In [1]:
import os
import sys
import pickle as pkl

repo_path = '/root/ADA2023/ada-2023-project-adacadabra2023/ada-2023-project-adacadabra2023' # CHANGE
sys.path.append(repo_path)

# from src.data.Data import load_data

In [2]:
dpath = os.path.join(repo_path, 'Data', 'Unified_ratings.pkl')
with open(dpath, 'rb') as f:
    df = pkl.load(f)
print(df.shape)

(10932248, 18)


# Columns filtration

In [3]:
df.columns

Index(['beer_name', 'beer_id', 'brewery_name', 'brewery_id', 'style', 'date',
       'rating', 'user_id', 'Procedence', 'country_user', 'country_brewery',
       'year', 'Season', 'z_score', 'exp_mean', 'diff_exp_mean', 'ith_rating',
       'detrend'],
      dtype='object')

In [4]:
df.head()

,beer_name,beer_id,brewery_name,brewery_id,style,date,rating,user_id,Procedence,country_user,country_brewery,year,Season,z_score,exp_mean,diff_exp_mean,ith_rating,detrend
0,Abita Andygator,3.0,Abita Brewing Company,1.0,Maibock / Helles Bock,2003-10-15 10:00:00,3.55,5605.0,BA,"United States, Illinois","United States, Louisiana",2003,Autumn,-0.272179,-0.272179,0.000000,1,-0.077426
1,Abita Andygator,3.0,Abita Brewing Company,1.0,Maibock / Helles Bock,2003-12-22 11:00:00,3.85,3474.0,BA,"United States, Massachusetts","United States, Louisiana",2003,Winter,0.168432,-0.051873,0.220306,2,0.362842
2,Abita Andygator,3.0,Abita Brewing Company,1.0,Maibock / Helles Bock,2004-04-22 10:00:00,2.39,2865.0,BA,"United States, Connecticut","United States, Louisiana",2004,Spring,-2.130088,-0.744612,-1.385477,3,-1.936023
3,Abita Andygator,3.0,Abita Brewing Company,1.0,Maibock / Helles Bock,2004-05-02 10:00:00,3.28,14960.0,BA,"United States, Texas","United States, Louisiana",2004,Spring,-0.757869,-0.747926,-0.009943,4,-0.564147
4,Abita Andygator,3.0,Abita Brewing Company,1.0,Maibock / Helles Bock,2004-10-14 10:00:00,3.17,3782.0,BA,"United States, Georgia","United States, Louisiana",2004,Autumn,-0.927469,-0.783835,-0.143635,5,-0.734091


In [5]:
filtered_df = df[['style','Season', 'country_brewery']]
filtered_df

,style,Season,country_brewery
0,Maibock / Helles Bock,Autumn,"United States, Louisiana"
1,Maibock / Helles Bock,Winter,"United States, Louisiana"
2,Maibock / Helles Bock,Spring,"United States, Louisiana"
3,Maibock / Helles Bock,Spring,"United States, Louisiana"
4,Maibock / Helles Bock,Autumn,"United States, Louisiana"
...,...,...,...
10932243,India Pale Ale (IPA),Summer,Poland
10932244,India Pale Ale (IPA),Summer,Poland
10932245,India Pale Ale (IPA),Summer,Poland
10932246,India Pale Ale (IPA),Summer,Poland


In [44]:
sty_country = filtered_df.groupby(['country_brewery', 'Season'], as_index=False).nunique()
sty_country


,country_brewery,Season,style
0,Albania,Autumn,3
1,Albania,Spring,3
2,Albania,Summer,3
3,Albania,Winter,3
4,Andorra,Autumn,1
...,...,...,...
859,Zambia,Winter,1
860,Zimbabwe,Autumn,1
861,Zimbabwe,Spring,1
862,Zimbabwe,Summer,1


In [49]:
select_countries = sty_country.groupby('country_brewery', as_index=False).min()
keep_countries = select_countries[select_countries['style'] >= 3]
valid_countries = keep_countries.country_brewery.values

In [53]:
filtered_countries = filtered_df[filtered_df['country_brewery'].isin(valid_countries)]

In [60]:
filtered_countries

(10932248, 3)

(10918870, 3)

In [61]:
top_4_beerstyles = filtered_countries.groupby(['country_brewery', 'Season'])['style'].value_counts().groupby(['country_brewery', 'Season']).head(3).reset_index(name='ratings')

In [77]:
top_4_beerstyles

,country_brewery,Season,style,ratings
0,Albania,Autumn,Pale Lager,61
1,Albania,Autumn,Dunkel/Tmavý,12
2,Albania,Autumn,Pilsener,12
3,Albania,Spring,Pale Lager,63
4,Albania,Spring,Pilsener,30
...,...,...,...,...
1891,Wales,Summer,American Double / Imperial IPA,1711
1892,Wales,Summer,American Amber / Red Ale,1627
1893,Wales,Winter,American Amber / Red Ale,4535
1894,Wales,Winter,India Pale Ale (IPA),1952


In [80]:
top_4_beerstyles.groupby(['country_brewery', 'Season'], as_index= False).agg(lambda x: x.tolist())


,country_brewery,Season,style,ratings
0,Albania,Autumn,"[ Pale Lager, Dunkel/Tmavý, Pilsener]","[61, 12, 12]"
1,Albania,Spring,"[ Pale Lager, Pilsener, Dunkel/Tmavý]","[63, 30, 16]"
2,Albania,Summer,"[ Pale Lager, Pilsener, Dunkel/Tmavý]","[92, 36, 32]"
3,Albania,Winter,"[ Pale Lager, Pilsener, Dunkel/Tmavý]","[48, 16, 5]"
4,Argentina,Autumn,"[ Pale Lager, American Adjunct Lager, America...","[217, 89, 40]"
...,...,...,...,...
627,Virgin Islands (U.S.),Winter,"[ India Pale Ale (IPA), Amber Ale, Fruit Beer]","[55, 43, 36]"
628,Wales,Autumn,"[India Pale Ale (IPA), American Amber / Red Al...","[2539, 1542, 1455]"
629,Wales,Spring,"[American Amber / Red Ale, India Pale Ale (IPA...","[2876, 1733, 1305]"
630,Wales,Summer,"[India Pale Ale (IPA), American Double / Imper...","[2716, 1711, 1627]"
